# CRN/FBA composite with Vivarium

## Overview
This notebook combines a chemical reaction network model with a flux balance analysis model, and a physics enging

In [1]:
#@title install vivarium libraries

#Uncomment in Colab Below

"""
%%capture
!pip install git+https://github.com/vivarium-collective/vivarium-notebooks
!pip install vivarium-core==0.1.5
!pip install vivarium-cobra==0.0.6
!pip install vivarium-pymunk==0.0.2
!pip install vivarium-bioscrape
"""

In [2]:
#@title imports
import numpy as np
import pylab as plt

from vivarium_cobra.processes.dynamic_fba import (
    DynamicFBA, get_iAF1260b_config, print_growth)
from vivarium_bioscrape.processes.bioscrape import (
    Bioscrape
)

# process/composition interface
from vivarium.core.process import (
    Process, Deriver, Composite,
)

# helper functions for composition
from vivarium.core.composition import (
    simulate_process_in_experiment,
    simulate_compartment_in_experiment,
    compartment_in_experiment,
    compose_experiment,
    FACTORY_KEY,
)

# other vivarium imports
from vivarium.core.experiment import Experiment, pp
from vivarium.library.units import units

# vivarium plotting functions
from vivarium.plots.simulation_output import plot_simulation_output
from vivarium.plots.agents_multigen import plot_agents_multigen
from vivarium.plots.topology import plot_compartment_topology

In [3]:
config =get_iAF1260b_config()
print(config)

{'model_path': 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\vivarium_cobra\\bigg_models\\iAF1260b.json'}


# Flux Balance Analysis with COBRA

 * This process is a wrapper for the popular FBA solver [COBRA](https://cobrapy.readthedocs.io/en/latest/).
 * Metabolism is initialized with a [BiGG metabolic model](http://bigg.ucsd.edu) called *iAF1260b*, which has 2382 reactions, 1261 genes, and 166 metabolites.

In [4]:
# plotting function for metabolism output
def plot_metabolism(data, ncol=2):
    original_fontsize = plt.rcParams['font.size']
    plt.rcParams.update({'font.size': 9})
    
    # initialize subplots
    n_rows = 2
    n_cols = 2
    fig = plt.figure(figsize=(n_cols * 7, n_rows * 3))
    grid = plt.GridSpec(n_rows, n_cols)
    
    time_vec = data['time']
    
    # mass
    ax = fig.add_subplot(grid[0, 0])  
    ax.plot(time_vec, data['global'][('mass', 'femtogram')], label = 'mass')
    ax.set_title('total compartment mass (fg)')
    ax.set_xlabel('time (sec)')
#     ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), ncol=ncol)
    
    # external
    ax = fig.add_subplot(grid[0, 1])  
    for mol_id, series in data['external'].items():
        if sum(series) != 0.0:
            ax.plot(time_vec, series, label = mol_id)
    ax.set_title('external concentrations (log)')
    ax.set_yscale('log')
    ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), ncol=ncol)
    
    # internal
    ax = fig.add_subplot(grid[1, 1])  
    for mol_id, series in data['internal_counts'].items():
        if sum(series) != 0.0:
            ax.plot(time_vec, series, label = mol_id)
    ax.set_title('internal molecule counts (log)')
    ax.set_xlabel('time (sec)')
    ax.set_yscale('log')
    fig.tight_layout()
    plt.rcParams.update({'font.size': original_fontsize})

In [ ]:
# get the configuration for the iAF1260b BiGG model
config = get_iAF1260b_config()   

# load it into DynamicFBA
metabolism = DynamicFBA(config)

# get the model's initial state
initial_state = metabolism.initial_state({}) 

# run simulation
sim_settings = {
    'initial_state': initial_state,
    'total_time': 2500}
cobra_timeseries = simulate_process_in_experiment(metabolism, sim_settings)


Experiment ID: d5678634-5a8d-11eb-9c40-6c2b5976b3a5
Created: 01/19/2021 at 11:37:58


In [ ]:
print_growth(cobra_timeseries['global'])
plot_metabolism(cobra_timeseries)

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Chemical Reaction Network with SBML 

In [ ]:
time_step = 1
total_time = 2500

# initialize Bioscrape process
bioscrape_config = {
    'sbml_file': 'LacOperon_simple.xml',
    'time_step': time_step}
bioscrape_process = Bioscrape(bioscrape_config)

# initial state
initial_state = bioscrape_process.initial_state()

# run simulation
settings = {
    'total_time': total_time,
    'initial_state': initial_state,
    'display_info': False,
    'progress_bar': False}
bioscrape_timeseries = simulate_process_in_experiment(bioscrape_process, settings)

In [ ]:
plot_simulation_output(bioscrape_timeseries)

# Multibody physics with pymunk

In [ ]:
from vivarium_pymunk.processes.multibody_physics import Multibody

# TODO -- minimal Growth/Division example here

# CRN/FBA Composite

This also includes FluxDeriver and imported division processes

In [ ]:
def get_metabolism_config(
    volume=1e-5*units.L,
):
    config = get_iAF1260b_config()
    config.update({
        'bin_volume': volume})
    return config

In [ ]:
class FluxDeriver(Deriver):
    defaults = {'time_step': 1}

    def initial_state(self, config=None):
        return {}

    def ports_schema(self):
        return {
            'deltas': {
                '*': {'_default': 0.0}},
            'fluxes': {
                '*': {'_default': 0.0}}}
                
    def next_update(self, timestep, states):
        deltas = states['deltas']
        return {
            'fluxes': {
                reaction_id: delta/self.parameters['time_step']
                for reaction_id, delta in deltas.items()}}

In [ ]:
from vivarium.processes.divide_condition import DivideCondition
from vivarium.processes.meta_division import MetaDivision
    
class CRN_COBRA(Generator):
    name = 'crn_cobra'
    defaults = {
        'agent_id': np.random.randint(0, 100),
        'crn': {
            'sbml_file': '../paper/LacOperon_simple.xml',
        },
        'cobra': get_metabolism_config(),
        'divide_condition': {
            'threshold': 2000 * units.fg
        },
        'agents_path': ('..', '..', 'agents',),
        'daughter_path': tuple(),
        '_schema': {
            'cobra': {
                'flux_bounds': {
                    '*': {
                        '_emit': True
                    }
                }
            }
        }
    }

    def generate_processes(self, config):
        # TODO -- reaction names from CRN needs to be matched with COBRA

        processes = {
            'crn': Bioscrape(config['crn']),
            'cobra': DynamicFBA(config['cobra'])}

        # configure derivers
        flux_config = {
            'time_step': processes['crn'].local_timestep()}
        division_config = dict(
            daughter_path=config['daughter_path'],
            agent_id=config['agent_id'],
            compartment=self)

        derivers = {
            'flux_deriver': FluxDeriver(flux_config),
            #             'divide_condition': DivideCondition(config['divide_condition']),
            #             'globals_deriver': DeriveGlobals({}),
            #             'division': MetaDivision(division_config),
        }

        return {**processes, **derivers}

    def generate_topology(self, config):
        return {
            'crn': {
                'species': ('species',),
                'delta_species': ('delta_species',),
                'rates': ('rates',),
                'globals': ('globals',),
            },
            'cobra': {
                'internal_counts': ('internal_counts',),
                'external': ('external',),
                'exchanges': ('exchanges',),
                'reactions': ('reactions',),
                'flux_bounds': ('flux_bounds',),
                'global': ('global',),
            },
            'flux_deriver': {
                'deltas': ('delta_species',),  # TODO -- need to add units!
                'fluxes': ('flux_bounds_2',),
            },
            #             'globals_deriver': {
            #                 'global': ('boundary',)
            #             },
            #             'divide_condition': {
            #                 'variable': ('boundary', 'mass',),
            #                 'divide': ('boundary', 'divide',)
            #             },
            #             'division': {
            #                 'global': ('boundary',),
            #                 'agents': config['agents_path']
            #             }
        }

In [ ]:
composite = CRN_COBRA({})
fig = plot_compartment_topology(composite)

In [ ]:
# plotting function for metabolism output
def plot_composite(data, ncol=2):
    original_fontsize = plt.rcParams['font.size']
    plt.rcParams.update({'font.size': 9})
    
    # initialize subplots
    n_rows = 2
    n_cols = 2
    fig = plt.figure(figsize=(n_cols * 7, n_rows * 3))
    grid = plt.GridSpec(n_rows, n_cols)
    
    time_vec = data['time']
    
    # mass
    ax = fig.add_subplot(grid[0, 0])  
    ax.plot(time_vec, data['agents']['1']['global'][('mass', 'femtogram')], label = 'mass')
    ax.set_title('total compartment mass (fg)')
    ax.set_xlabel('time (sec)')
    ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), ncol=ncol)
    
    # external
    ax = fig.add_subplot(grid[0, 1])  
    for mol_id, series in data['agents']['1']['external'].items():
        if sum(series) != 0.0:
            ax.plot(time_vec, series, label = mol_id)
    ax.set_title('external concentrations (log)')
    ax.set_yscale('log')
    ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), ncol=ncol)
    
    # internal
    ax = fig.add_subplot(grid[1, 1])  
    for mol_id, series in data['agents']['1']['internal_counts'].items():
        if sum(series) != 0.0:
            ax.plot(time_vec, series, label = mol_id)
    ax.set_title('internal molecule counts (log)')
    ax.set_xlabel('time (sec)')
    ax.set_yscale('log')
    
    # layout
    fig.tight_layout()
    plt.rcParams.update({'font.size': original_fontsize})

## simulate CRN/FBA composite

In [ ]:
agent_id = '1'
time_step = 10
total_time = 2000
volume=1e-5 * units.L

# initialize composite
cobra_config = get_iAF1260b_config()
cobra_config.update({
    'time_step': time_step,
    'bin_volume': volume})
crn_config = {'time_step': time_step}

composite_config = {
    'agent_id': agent_id,
    'crn': crn_config,
    'cobra': cobra_config}

composite = CRN_COBRA(composite_config)

# get initial state
initial_state = {
    'agents': {
        '1': composite.initial_state()}}

# make the experiment
sim_settings = {
    'initial_state': initial_state,
    'outer_path': ('agents', agent_id,)}
composite_experiment = compartment_in_experiment(composite, sim_settings)

# run simulation
composite_experiment.update(total_time)

In [ ]:
# retrieve data and plot
composite_timeseries = composite_experiment.emitter.get_timeseries()
print_growth(composite_timeseries['agents']['1']['global'])
plot_composite(composite_timeseries)

In [ ]:
# plot output
plot_settings = {
    'remove_flat': True}
fig = plot_simulation_output(composite_timeseries, plot_settings)

# Simulate CRN/FBA in a Spatial Environment with pymunk
 * We here plug GrowDivide into a composite called Lattice, which has a multi-body physics process (MBP) and a spatial diffusion process (DFF).

In [ ]:
# Environment model with multi-body physics engine and discrete concentration diffusion
from vivarium_cell.composites.lattice import (
    Lattice,
    make_lattice_config)

# configure the lattice environment
lattice_bounds = [15, 15]
lattice_config = make_lattice_config(
    jitter_force=1e-5,
    bounds=lattice_bounds,
    n_bins=[10, 10],
    time_step=30)

# experiment settings to hide info from being plotted
experiment_settings = {'progress_bar': False, 'display_info': False}

In [ ]:
from vivarium.processes.divide_condition import DivideCondition
from vivarium.processes.meta_division import MetaDivision
from vivarium_cell.processes.derive_globals import DeriveGlobals

class ExponentialGrowth(Process):

    name = 'exponential_growth'
    defaults = {
        'growth_rate': 0.0005,
        'growth_noise': 1e-4}
    
    def __init__(self, parameters=None):
        super(ExponentialGrowth, self).__init__(parameters)
    
    def ports_schema(self):
        return {
            'port1': {
                'mass': {
                    '_default': 1000.0 * units.fg,
                    '_updater': 'set',
                    '_divider': 'split',
                    '_emit': True}}}
    

    def next_update(self, timestep, states):
        mass = states['port1']['mass']
        new_mass = mass * (
            np.exp(self.parameters['growth_rate'] +
                   np.random.normal(0,self.parameters['growth_noise'])
                  ) * timestep)
        return {
            'port1': {
                'mass': new_mass}}

class GrowDivide(Generator):
    name = 'grow_divide'
    defaults = {
        'agent_id': np.random.randint(0,100),
        'exponential_growth': {'growth_noise': 1e-3},
        'divide_condition': {'threshold': 2000 * units.fg},
        'agents_path': ('..', '..', 'agents',),
        'daughter_path': tuple()}

    def __init__(self, config):
        super(GrowDivide, self).__init__(config)
    
    def generate_processes(self, config):
        division_config = dict(
            daughter_path=config['daughter_path'], 
            agent_id=config['agent_id'], 
            compartment=self)
        return {
            'exponential_growth': ExponentialGrowth(config['exponential_growth']),
            'divide_condition': DivideCondition(config['divide_condition']),
            'globals_deriver': DeriveGlobals({}),
            'division': MetaDivision(division_config)}

    def generate_topology(self, config):
        return {
            'exponential_growth': {
                'port1': ('boundary',)},
            'globals_deriver': {
                'global': ('boundary',)},
            'divide_condition': {
                'variable': ('boundary', 'mass',),
                'divide': ('boundary', 'divide',)},
            'division': {
                'global': ('boundary',),
                'agents': config['agents_path']}}

In [ ]:
# GrowDivide requires an 'agent_id' for the division process to track agents
agent_id = '0'
gd_params = {
    'agent_id': agent_id,
    'growth_rate': {
        'growth_rate': 0.001}}

initial_state = {
    'agents': {
        agent_id: {
            'mass': 1000 * units.fg}}}

gd_sim_settings = {
    'initial_state': initial_state,
    'outer_path': ('agents', agent_id),
    'display_info': False,
    'progress_bar': False}

gd_plot_settings = {
    'column_width': 12,
    'include_paths': [
        ('boundary', 'mass')]}

In [ ]:
# declare the hierarchy, with a GrowDivide agent within an 'agents' store
hierarchy = {
    GENERATORS_KEY: {
        'type': Lattice,
        'config': lattice_config},
    'agents': {
        agent_id: {
            GENERATORS_KEY: {
                'type': GrowDivide,
                'config': gd_params}}}}

# configure experiment with helper function compose_experiment
experiment = compose_experiment(
    hierarchy=hierarchy, 
    settings=experiment_settings)

# run the experiment
experiment.update(800)
lattice_output = experiment.emitter.get_data()

In [ ]:
from vivarium_cell.plots.multibody_physics import plot_tags, plot_snapshots

# extract data
agents = {time: time_data['agents'] for time, time_data in lattice_output.items()}
fields = {time: time_data['fields'] for time, time_data in lattice_output.items()}
plot_data = {
    'agents': agents,
    'fields': fields,
    'config': {'bounds': lattice_bounds},
}
plot_config = {
    'include_fields': [],
    'n_snapshots': 5,
    'default_font_size': 36
}

plot_snapshots(plot_data, plot_config)